<a href="https://colab.research.google.com/github/rileyafox/llama-code/blob/main/LLama_Code_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers datasets accelerate trl peft bitsandbytes unsloth evaluate rouge_score sacrebleu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.0/277.0 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.4/147.4 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 28.0 MB/s eta 0:00

In [ ]:
import os
import re
import subprocess
import tempfile
from collections import Counter

import torch
from huggingface_hub import login
from google.colab import userdata, drive
from datasets import load_from_disk, load_dataset
from transformers import TrainingArguments, Trainer
from unsloth import FastLanguageModel

<ipython-input-2-1458664192>:12: UserWarning: WARNING: Unsloth should be imported before transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
# -----------------------------------------------------------------------------
# 1. AUTH & DRIVE SET‑UP
# -----------------------------------------------------------------------------
login(token=userdata.get("HF_TOKEN"))
drive.mount("/content/drive")

DRIVE_BASE = "/content/drive/MyDrive"
OUTPUT_DIR = f"{DRIVE_BASE}/llama-opencode"
TOKENIZED_TRAIN_PATH = f"{DRIVE_BASE}/opencode_tokenized_train"
TOKENIZED_TEST_PATH = f"{DRIVE_BASE}/opencode_tokenized_test"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# -----------------------------------------------------------------------------
# 2. HYPER‑PARAMETERS (EDIT IN ONE SPOT)
# -----------------------------------------------------------------------------
MAX_LENGTH = 2048
BASE_MODEL = "unsloth/llama-2-13b-bnb-4bit"
EPOCHS      = 1
LR          = 2e-4

# LoRA
LORA_R          = 64
LORA_ALPHA      = 64
LORA_DROPOUT    = 0.1

# Generation / evaluation
GEN_K           = 10             # candidates per problem (pass@k)
TEMPERATURE     = 0.7
TOP_P           = 0.95
MAX_NEW_TOKENS  = 256

KEEP_COLS         = ["input_ids", "attention_mask", "labels"]  # only these go to Trainer


In [ ]:
# -----------------------------------------------------------------------------
# 3. HELPERS
# -----------------------------------------------------------------------------

def get_latest_checkpoint(path: str):
    if not os.path.exists(path):
        return None
    ckpts = [d for d in os.listdir(path) if d.startswith("checkpoint-")]
    if not ckpts:
        return None
    latest = sorted(ckpts, key=lambda x: int(x.split("-")[-1]))[-1]
    return os.path.join(path, latest)

RESUME_FROM = get_latest_checkpoint(OUTPUT_DIR)
print("Resuming from:", RESUME_FROM or "scratch")


Resuming from: scratch


In [ ]:
# -----------------------------------------------------------------------------
# 4. MODEL + TOKENISER
# -----------------------------------------------------------------------------
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = RESUME_FROM or BASE_MODEL,
    max_seq_length = MAX_LENGTH,
    dtype = torch.float16,
    load_in_4bit = True,
)

tokenizer.pad_token = tokenizer.eos_token

model = FastLanguageModel.get_peft_model(
    model,
    r                  = LORA_R,
    lora_alpha         = LORA_ALPHA,
    lora_dropout       = LORA_DROPOUT,
    bias               = "none",
    use_gradient_checkpointing = True,
    random_state       = 42,
    use_rslora         = False,
    loftq_config       = None,
)


==((====))==  Unsloth 2025.6.2: Fast Llama patching. Transformers: 4.52.4.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/7.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/948 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.1.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.6.2 patched 40 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [ ]:
# -----------------------------------------------------------------------------
# 5. DATASET LOADING / TOKENISATION (with FIM option)
# -----------------------------------------------------------------------------

def is_within_length(example):
    combined = f"<problem>\n{example['input'].strip()}\n<solution>\n{example['solution'].strip()}"
    return len(tokenizer(combined).input_ids) <= MAX_LENGTH


def tokenize(batch):
    full = [f"<problem>\n{i.strip()}\n<solution>\n{s.strip()}" for i, s in zip(batch["input"], batch["solution"])]
    toks = tokenizer(full, truncation=True, padding="max_length", max_length=MAX_LENGTH)
    toks["labels"] = toks["input_ids"].copy()
    return toks

if os.path.exists(TOKENIZED_TRAIN_PATH) and os.path.exists(TOKENIZED_TEST_PATH):
    print("Loading tokenised splits …")
    tokenized_train = load_from_disk(TOKENIZED_TRAIN_PATH)
    tokenized_test  = load_from_disk(TOKENIZED_TEST_PATH)
else:
    print("Tokenising raw OpenCodeReasoning split …")
    ds = load_dataset("nvidia/OpenCodeReasoning", "split_0")["split_0"]
    raw_split = ds.train_test_split(test_size=0.01)
    filtered_train = raw_split["train"].filter(is_within_length)
    filtered_test  = raw_split["test"].filter(is_within_length)
    tokenized_train = filtered_train.map(tokenize, batched=True)
    tokenized_test  = filtered_test .map(tokenize, batched=True)
    tokenized_train.save_to_disk(TOKENIZED_TRAIN_PATH)
    tokenized_test .save_to_disk(TOKENIZED_TEST_PATH)

# 🔑  Drop non-tensor columns to stop Trainer ValueError
for ds_name, ds in {"train": tokenized_train, "test": tokenized_test}.items():
    drop_cols = [c for c in ds.column_names if c not in KEEP_COLS]
    if drop_cols:
        print(f"• Removing non-tensor columns from {ds_name}:", drop_cols)
        if ds_name == "train":
            tokenized_train = ds.remove_columns(drop_cols)
        else:
            tokenized_test  = ds.remove_columns(drop_cols)

print("Example tokenised row:", tokenized_train[0])

# Clamp to a small subset if debugging
# tokenized_train = tokenized_train.select(range(25000))


Loading tokenised splits …


Loading dataset from disk:   0%|          | 0/66 [00:00<?, ?it/s]

• Removing non-tensor columns from train: ['id', 'input', 'output', 'source', 'license', 'dataset', 'split', 'difficulty', 'solution']
• Removing non-tensor columns from test: ['id', 'input', 'output', 'source', 'license', 'dataset', 'split', 'difficulty', 'solution']
Example tokenised row: {'input_ids': [1, 529, 17199, 29958, 13, 3492, 1369, 411, 278, 1353, 29871, 29900, 322, 366, 864, 304, 6159, 278, 1353, 405, 29889, 13, 13, 3492, 508, 1735, 278, 1353, 29892, 5146, 292, 263, 3058, 5253, 310, 1302, 1144, 29892, 411, 278, 1494, 6931, 29901, 13, 13, 29930, 9683, 666, 368, 278, 1353, 491, 29871, 29906, 29892, 5146, 292, 319, 1302, 1144, 29889, 13, 29930, 9683, 666, 368, 278, 1353, 491, 29871, 29941, 29892, 5146, 292, 350, 1302, 1144, 29889, 13, 29930, 9683, 666, 368, 278, 1353, 491, 29871, 29945, 29892, 5146, 292, 315, 1302, 1144, 29889, 13, 29930, 512, 1037, 559, 470, 23806, 278, 1353, 491, 29871, 29896, 29892, 5146, 292, 360, 1302, 1144, 29889, 13, 13, 13, 13, 3492, 508, 2189, 1438, 6

In [ ]:
# -----------------------------------------------------------------------------
# 6. TRAINING ARGS
# -----------------------------------------------------------------------------
training_args = TrainingArguments(
    output_dir              = OUTPUT_DIR,
    num_train_epochs        = EPOCHS,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size  = 8,
    gradient_accumulation_steps = 4,
    learning_rate           = LR,
    lr_scheduler_type       = "cosine",
    warmup_ratio            = 0.03,
    logging_steps           = 50,
    save_strategy           = "steps",
    save_steps              = 100,
    save_total_limit        = 2,
    fp16                    = True,
    report_to               = "none",
    remove_unused_columns   = False,
)

trainer = Trainer(
    model          = model,
    args           = training_args,
    train_dataset  = tokenized_train,
    eval_dataset   = None,
    tokenizer      = tokenizer,
)

trainer.train(resume_from_checkpoint = RESUME_FROM)
model.save_pretrained(f"{DRIVE_BASE}/llama-finetuned")

<ipython-input-21-258909275>:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 540,594 | Num Epochs = 1 | Total steps = 16,894
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 4 x 1) = 32
 "-____-"     Trainable parameters = 250,347,520/13,000,000,000 (1.93% trained)


Step,Training Loss
50,1.530700
100,0.496900
150,0.403700
200,0.390300
250,0.381000
300,0.376800
350,0.368300
400,0.364800
450,0.361200
500,0.351800


Step,Training Loss
50,1.530700
100,0.496900
150,0.403700
200,0.390300
250,0.381000
300,0.376800
350,0.368300
400,0.364800
450,0.361200
500,0.351800


In [ ]:
# -----------------------------------------------------------------------------
# 7. EVALUATION – PASS@k WITH SAMPLING
# -----------------------------------------------------------------------------

ev_ds = load_dataset("nvidia/OpenCodeReasoning", "split_0")["split_0"].train_test_split(test_size=0.01)["test"].select(range(100))
cf_ds = ev_ds.filter(lambda e: e["source"] == "codeforces")

finetuned_model, finetuned_tokenizer = FastLanguageModel.from_pretrained(
    model_name      = f"{DRIVE_BASE}/llama-finetuned",
    max_seq_length  = MAX_LENGTH,
    dtype           = torch.float16,
    load_in_4bit    = True,
)
finetuned_model.eval(); finetuned_tokenizer.pad_token = finetuned_tokenizer.eos_token


def format_prompt(problem: str) -> str:
    return (
        "<|system|>\nYou are an expert competitive‑programming assistant.\n<|user|>\n### Problem\n" + problem.strip() +
        "\n<|assistant|>\n```python\n"
    )


def generate_k(model, tokenizer, prompt: str, k: int = GEN_K):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model.generate(
            **inputs,
            do_sample=True,
            temperature=TEMPERATURE,
            top_p=TOP_P,
            max_new_tokens=MAX_NEW_TOKENS,
            num_return_sequences=k,
            eos_token_id=tokenizer.eos_token_id,
        )
    dec = tokenizer.batch_decode(out, skip_special_tokens=True)
    # keep only code after first python fence
    return [seg.split("```python\n",1)[-1] for seg in dec]


def execute_code(code: str, inp: str, timeout: int = 5):
    with tempfile.NamedTemporaryFile(mode="w", suffix=".py", delete=False) as f:
        f.write(code)
        path = f.name
    try:
        p = subprocess.run(["python3", path], input=inp, text=True, capture_output=True, timeout=timeout)
        return p.stdout.strip(), p.stderr.strip(), p.returncode
    except subprocess.TimeoutExpired:
        return "", "Timeout", -1
    finally:
        os.remove(path)


def normalise(s: str):
    return "\n".join(l.strip().lower() for l in s.splitlines() if l.strip())


pass_any = 0
reason_counter = Counter()

for ex in cf_ds:
    prompt = ex["input"]
    inp    = re.search(r"Examples.*?Input\s*\n(.*?)\n+Output", prompt, re.S).group(1).strip()
    exp    = re.search(r"Output\s*\n(.*?)($|\n{2,})", prompt, re.S).group(1).strip()

    candidates = generate_k(finetuned_model, finetuned_tokenizer, format_prompt(prompt))
    solved = False
    for code in candidates:
        out, err, rc = execute_code(code, inp)
        if rc == 0 and normalise(out) == normalise(exp):
            solved = True
            break
    if solved:
        pass_any += 1
    else:
        reason_counter.update(["fail"])

print(f"\nPASS@{GEN_K}: {pass_any}/{len(cf_ds)} = {pass_any/len(cf_ds):.2%}")
